In [1]:
import pandas as pd

df = pd.read_csv('golden_annotated_merged.csv')

In [2]:
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split

# Оставляем только казахские (если это флаг-модель)
df = df[df["is_kazakh_model"] == True].copy()

# Убедимся, что таргеты действительно bool → int
label_cols = ["toxic", "obscene", "threat", "insult", "hate"]
for c in label_cols:
    df[c] = df[c].astype(int)

print(df[label_cols].sum())   # просто посмотреть распределение

# train/val/test split
train_df, temp_df = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    stratify=df["is_toxic"].astype(int)  # можно стратифицировать по общему флагу
)
valid_df, test_df = train_test_split(
    temp_df,
    test_size=0.5,
    random_state=42,
    stratify=temp_df["is_toxic"].astype(int)
)

train_ds = Dataset.from_pandas(train_df.reset_index(drop=True))
valid_ds = Dataset.from_pandas(valid_df.reset_index(drop=True))
test_ds  = Dataset.from_pandas(test_df.reset_index(drop=True))


C:\Users\marin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


toxic      1104
obscene     195
threat      152
insult      780
hate        135
dtype: int64


In [3]:
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)
import torch
import numpy as np
from sklearn.metrics import f1_score, accuracy_score, precision_recall_fscore_support

# MODEL_NAME = "cointegrated/rubert-tiny-toxicity"
# MODEL_NAME = "DeepPavlov/rubert-base-cased"
MODEL_NAME = "kz-transformers/kaz-roberta-conversational"


tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(label_cols),              # 5
    problem_type="multi_label_classification"
)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at kz-transformers/kaz-roberta-conversational and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
def tokenize_batch(batch):
    return tokenizer(
        batch["text_original"],
        padding="max_length",
        truncation=True,
        max_length=128,
    )

import numpy as np

label_cols = ["toxic", "obscene", "threat", "insult", "hate"]

def add_labels(batch):
    # batch[c] — это список значений по батчу, собираем в матрицу
    labels = np.vstack([batch[c] for c in label_cols]).T.astype("float32")
    # вернем только поле labels, чтобы не дублировать
    return {"labels": labels}


train_ds_tok = train_ds.map(tokenize_batch, batched=True)
valid_ds_tok = valid_ds.map(tokenize_batch, batched=True)
test_ds_tok  = test_ds.map(tokenize_batch, batched=True)

train_ds_tok = train_ds_tok.map(add_labels, batched=True)
valid_ds_tok = valid_ds_tok.map(add_labels, batched=True)
test_ds_tok  = test_ds_tok.map(add_labels, batched=True)


cols = ["input_ids", "attention_mask", "labels"]

train_ds_tok.set_format(
    type="torch",
    columns=cols,
)
valid_ds_tok.set_format(
    type="torch",
    columns=cols,
)
test_ds_tok.set_format(
    type="torch",
    columns=cols,
)


Map: 100%|██████████| 301/301 [00:00<00:00, 62069.10 examples/s]


In [5]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = 1 / (1 + np.exp(-logits))  # sigmoid

    # порог 0.5, можно потом поиграть (0.3–0.7)
    y_pred = (probs >= 0.5).astype(int)
    y_true = labels

    # micro/macro по всем лейблам
    micro_p, micro_r, micro_f1, _ = precision_recall_fscore_support(
        y_true, y_pred, average="micro", zero_division=0
    )
    macro_p, macro_r, macro_f1, _ = precision_recall_fscore_support(
        y_true, y_pred, average="macro", zero_division=0
    )

    # можно ещё посчитать per-label f1 по каждому типу токсичности:
    per_label_f1 = f1_score(y_true, y_pred, average=None, zero_division=0)

    metrics = {
        "micro_f1": micro_f1,
        "micro_precision": micro_p,
        "micro_recall": micro_r,
        "macro_f1": macro_f1,
        "macro_precision": macro_p,
        "macro_recall": macro_r,
    }
    # добавим f1 по каждому лейблу с префиксом
    for i, name in enumerate(label_cols):
        metrics[f"f1_{name}"] = per_label_f1[i]

    return metrics


In [7]:
from transformers import EarlyStoppingCallback, TrainingArguments
import torch
from torch.nn import BCEWithLogitsLoss
from transformers import Trainer

# ========== TrainingArguments ==========
training_args = TrainingArguments(
    output_dir="./rubert_tiny_kaz_multi",
    eval_strategy="epoch",        # FIXED
    save_strategy="epoch",
    logging_steps=50,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="micro_f1",
    fp16=torch.cuda.is_available(),
    save_total_limit=3
)

# ========== Weighted Trainer ==========
import torch
from torch.nn import BCEWithLogitsLoss
from transformers import Trainer

class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        # достаём labels и убираем их из inputs
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits

        # pos_weight ты уже положил в trainer.pos_weight = pos_weight
        loss_fct = BCEWithLogitsLoss(pos_weight=self.pos_weight.to(logits.device))
        loss = loss_fct(logits, labels)

        if return_outputs:
            return loss, outputs
        return loss


# ========== Compute class weights ==========
label_counts = df[label_cols].sum()
neg_counts = len(df) - label_counts
pos_weight = (neg_counts / label_counts).values

pos_weight = torch.tensor(pos_weight, dtype=torch.float32)
print("pos_weight =", pos_weight)

# safety check
assert len(pos_weight) == model.config.num_labels, "Размер pos_weight != num_labels"

# ========== Trainer ==========
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_ds_tok,
    eval_dataset=valid_ds_tok,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
)

# attach weights
trainer.pos_weight = pos_weight

# ========== Train ==========
trainer.train()

# ========== Evaluate ==========
print(trainer.evaluate(test_ds_tok))

# ========== Save ==========
trainer.save_model("./rubert_tiny_kaz_multi_best")
tokenizer.save_pretrained("./rubert_tiny_kaz_multi_best")


pos_weight = tensor([ 1.7210, 14.4051, 18.7632,  2.8513, 21.2519])


C:\Users\marin\AppData\Local\Temp\ipykernel_5012\1880180702.py:56: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedTrainer.__init__`. Use `processing_class` instead.
  trainer = WeightedTrainer(


Epoch,Training Loss,Validation Loss,Micro F1,Micro Precision,Micro Recall,Macro F1,Macro Precision,Macro Recall,F1 Toxic,F1 Obscene,F1 Threat,F1 Insult,F1 Hate
1,0.844400,0.812360,0.536667,0.437500,0.693966,0.453647,0.362696,0.713058,0.672566,0.342857,0.385965,0.566845,0.300000
2,0.623300,0.692381,0.543885,0.408207,0.814655,0.458450,0.347849,0.864026,0.705394,0.320000,0.342105,0.627451,0.297297
3,0.443400,0.761440,0.623616,0.545161,0.728448,0.543943,0.457785,0.717767,0.717489,0.400000,0.555556,0.655367,0.391304
4,0.306900,0.904605,0.654902,0.600719,0.719828,0.570518,0.529686,0.624867,0.724138,0.470588,0.580645,0.648649,0.428571
5,0.185900,1.052896,0.690909,0.650190,0.737069,0.592322,0.577571,0.615699,0.768559,0.545455,0.583333,0.689266,0.375000
6,0.129900,1.157372,0.669439,0.646586,0.693966,0.605574,0.592941,0.625010,0.737778,0.578947,0.592593,0.638554,0.480000
7,0.107900,1.272083,0.666667,0.652893,0.681034,0.580018,0.586294,0.577641,0.735426,0.457143,0.583333,0.662651,0.461538


{'eval_loss': 1.1211813688278198, 'eval_micro_f1': 0.63671875, 'eval_micro_precision': 0.5821428571428572, 'eval_micro_recall': 0.7025862068965517, 'eval_macro_f1': 0.45775032987127906, 'eval_macro_precision': 0.4149801992958719, 'eval_macro_recall': 0.5262712929379596, 'eval_f1_toxic': 0.7257383966244726, 'eval_f1_obscene': 0.2, 'eval_f1_threat': 0.34782608695652173, 'eval_f1_insult': 0.6951871657754011, 'eval_f1_hate': 0.32, 'eval_runtime': 0.1513, 'eval_samples_per_second': 1989.122, 'eval_steps_per_second': 66.084, 'epoch': 7.0}


('./rubert_tiny_kaz_multi_best\\tokenizer_config.json',
 './rubert_tiny_kaz_multi_best\\special_tokens_map.json',
 './rubert_tiny_kaz_multi_best\\vocab.json',
 './rubert_tiny_kaz_multi_best\\merges.txt',
 './rubert_tiny_kaz_multi_best\\added_tokens.json',
 './rubert_tiny_kaz_multi_best\\tokenizer.json')

In [8]:
MODEL_DIR = "./kaz_roberta_toxic_kz"  # любая папка

model.save_pretrained(MODEL_DIR)
tokenizer.save_pretrained(MODEL_DIR)


('./kaz_roberta_toxic_kz\\tokenizer_config.json',
 './kaz_roberta_toxic_kz\\special_tokens_map.json',
 './kaz_roberta_toxic_kz\\vocab.json',
 './kaz_roberta_toxic_kz\\merges.txt',
 './kaz_roberta_toxic_kz\\added_tokens.json',
 './kaz_roberta_toxic_kz\\tokenizer.json')